In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
print(sys.version_info)

sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)


In [ ]:
!pip install rotation-forest
import numpy as np
import os
import tarfile
import zipfile
import email
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import re
from collections import Counter
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from bs4 import Comment
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from rotation_forest import RotationForestClassifier
from sklearn.tree import DecisionTreeClassifier
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
rf = RotationForestClassifier(max_depth=50, n_estimators=5)
svm = SVC(max_iter=10000)
cert = DecisionTreeClassifier()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
"""
Unzip datasets
"""

ceas = tarfile.open('/content/drive/My Drive/Colab Notebooks/Coursework/datasets/ceas08-1.tgz', 'r:gz')
trec = tarfile.open('/content/drive/My Drive/Colab Notebooks/Coursework/datasets/trec07p.tgz', 'r:gz')
enron = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Coursework/datasets/archive.zip', 'r')
ceas.extractall()
trec.extractall()
enron.extractall()
spamassassin = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Coursework/datasets/spamassassin.zip', 'r')
spamassassin.extractall()

In [ ]:
"""
Extract emails from datasets TREC07 and CEAS08
"""
def extract_emails_tc(index_path, dataset_tag):
  index_file = open(index_path, 'r')
  index_list = index_file.readlines()
  email_texts = np.empty(len(index_list), dtype=object)
  email_classes = np.zeros(len(index_list), dtype=bool)
  service_info = {}
  with tf.device('/device:GPU:0'):
    for i, index in enumerate(index_list):
      info = index.strip().split()
      email_classes[i] = True if 'spam' in info[0].lower() else False
      curr_email = open(os.path.normpath(os.path.join(os.path.dirname(index_path), info[1])), encoding='latin-1')
      #line = curr_email.readline()
      #if dataset_tag == 'ceas':
        #while not 'x-ceas-tracking:' in line:
          #line = curr_email.readline()
      #elif dataset_tag == 'trec':
        #while not line == '\n':
          #line = curr_email.readline()
      email_texts[i] = curr_email.read()
  return email_texts, email_classes

In [ ]:
ceas_texts, ceas_classes = extract_emails_tc('/content/ceas08-1/full-pretrain-nofeedback/index', 'ceas')
print(len(ceas_texts))

140772


In [ ]:
trec_texts, trec_classes = extract_emails_tc('/content/trec07p/full/index', 'trec')
print(len(trec_texts))

75419


In [ ]:
def extract_emails_enron(data_dir):
  spamdir = os.path.join(data_dir, 'spam')
  hamdir = os.path.join(data_dir, 'ham')
  nspam = len(os.listdir(spamdir))
  nham = len(os.listdir(hamdir))
  enron_classes = np.concatenate((np.ones(nspam), np.zeros(nham))).astype(bool)
  enron_texts = np.empty(nspam + nham, dtype=object)
  for i, filename in enumerate(os.listdir(spamdir)):
    curr_email = open(os.path.join(spamdir, filename), encoding='latin-1')
    enron_texts[i] = curr_email.read()
  for i, filename in enumerate(os.listdir(hamdir)):
    curr_email = open(os.path.join(hamdir, filename), encoding='latin-1')
    enron_texts[i + nspam] = curr_email.read()
  return enron_texts, enron_classes

In [ ]:
en1t, en1c = extract_emails_enron('/content/enron1')
en2t, en2c = extract_emails_enron('/content/enron2')
en3t, en3c = extract_emails_enron('/content/enron3')
en4t, en4c = extract_emails_enron('/content/enron4')
en5t, en5c = extract_emails_enron('/content/enron5')
en6t, en6c = extract_emails_enron('/content/enron6')
print(en1t)

['Subject: get prescri ) ption d ) rugs to your door !\n'
 'Subject: i tried , it worked infinitesimal ingest\nyour dog derm\ni begin by taking . i shall find scholars later to demonstrate my perfect right . - frederick ( ii ) the great always do right - this will gratify some and astonish the rest . - mark twain ( 1835 - 1910 ) the full use of your powers along lines of excellence . - definition ofhappiness by john f . kennedy ( 1917 - 1963 )\n'
 "Subject: i ' ll never stop loving you\nat what age is it easiest to fall in love . ?\nit doesnt matterif you can meet the right person .\nbut the question iswhere do you find the right person .\nclick on the link below and discover where thousands of people have fallen in love and its free .\nclick here :\nhttp : / / keeponlovinme . com / search / ? oc = 5303\n"
 ...
 'Subject: vastar resources , inc .\ngary , production from the high island larger block a - 1 # 2 commenced on\nsaturday at 2 : 00 p . m . at about 6 , 500 gross . carlos expec

In [ ]:
def extract_emails_spamassassin():
  spamdir = '/content/spam_2/spam_2'
  ehamdir = '/content/easy_ham/easy_ham'
  hhamdir = '/content/hard_ham/hard_ham'
  nspam = len(os.listdir(spamdir))
  neham = len(os.listdir(ehamdir))
  nhham = len(os.listdir(hhamdir))
  sa_classes = np.concatenate((np.ones(nspam), np.zeros(neham + nhham))).astype(bool)
  sa_texts = np.empty(nspam + nhham + neham, dtype=object)
  for i, filename in enumerate(os.listdir(spamdir)):
    curr_email = open(os.path.join(spamdir, filename), encoding='latin-1')
    sa_texts[i] = curr_email.read()
  for i, filename in enumerate(os.listdir(ehamdir)):
    curr_email = open(os.path.join(ehamdir, filename), encoding='latin-1')
    sa_texts[i + nspam] = curr_email.read()
  for i, filename in enumerate(os.listdir(hhamdir)):
    curr_email = open(os.path.join(hhamdir, filename), encoding='latin-1')
    sa_texts[i + nspam + neham] = curr_email.read()
  return sa_texts, sa_classes

In [ ]:
sa_texts, sa_classes = extract_emails_spamassassin()
print(sa_texts)

['From scott@hetnet.nl  Fri Aug  9 07:11:07 2002\nReturn-Path: <scott@hetnet.nl>\nDelivered-To: yyyy@localhost.netnoteinc.com\nReceived: from localhost (localhost [127.0.0.1])\n\tby phobos.labs.netnoteinc.com (Postfix) with ESMTP id ABD99440CF\n\tfor <jm@localhost>; Fri,  9 Aug 2002 02:11:07 -0400 (EDT)\nReceived: from mail.webnote.net [193.120.211.219]\n\tby localhost with POP3 (fetchmail-5.9.0)\n\tfor jm@localhost (single-drop); Fri, 09 Aug 2002 07:11:07 +0100 (IST)\nReceived: from smtp.easydns.com (smtp.easydns.com [205.210.42.30])\n\tby webnote.net (8.9.3/8.9.3) with ESMTP id HAA03799;\n\tFri, 9 Aug 2002 07:06:53 +0100\nFrom: scott@hetnet.nl\nReceived: from hetnet.nl (dsl-200-67-211-23.prodigy.net.mx [200.67.211.23])\n\tby smtp.easydns.com (Postfix) with SMTP\n\tid 65FEB2F44A; Fri,  9 Aug 2002 02:05:47 -0400 (EDT)\nReply-To: <scott@hetnet.nl>\nMessage-ID: <006c66d63bed$3826c0e8$5be04cb4@sqkorj>\nTo: micon@hotmail.com\nSubject: HELP WANTED. WORK FROM HOME. FREE INFO\nMiME-Version: 1

In [ ]:
def count_email_types(arr):
  parts = {}
  parts_decoded = {}
  for text in arr:
    msg = email.message_from_string(text)
    if msg.is_multipart():
      for part in msg.walk():
        parts[part.get_content_type()] = part.get_payload()
        parts_decoded[part.get_content_type()] = part.get_payload(decode=True)
    else:
        parts[msg.get_content_type()] = part.get_payload()
        parts_decoded[msg.get_content_type()] = msg.get_payload(decode=True)
  return parts, parts_decoded

with tf.device('/device:GPU:0'):
  trec_types, trec_decoded = count_email_types(trec_texts)

In [ ]:
print(trec_types['message/rfc822']

AttributeError: ignored

In [ ]:
def tag_visible(element):
  if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]'] or isinstance(element, Comment):
      return False
  else:
      return True

def process_html(txt):
  soup = BeautifulSoup(txt, 'html.parser')
  texts = soup.findAll(text=True)
  visible_texts = filter(tag_visible, texts)
  return ' '.join(visible_texts) 

def process_part(part):
  if part.get_content_type() == 'text/':
    return part.get_payload(decode=True).decode('latin-1')
  if part.get_content_maintype() == "text" or part.get_content_maintype() == "plain":
    txt = part.get_payload(decode=True).decode('latin-1')
    if bool(BeautifulSoup(txt, "html.parser").find()):
      return process_html(txt)
    elif part.get_content_subtype() == 'plain' or part.get_content_subtype() == 'enriched' or part.get_content_subtype() == 'text':
      return txt
    else:
      return ''
  else:
    return ''
    

def process_email(email):
  email_txt = ''
  if email.is_multipart():
    for part in email.walk():
      if part.get_content_maintype() == "multipart" or part.get_content_maintype() == 'image':
        continue
      email_txt += process_part(part)
  else:
    email_txt = process_part(email)
  return email_txt

def get_processed_arr(texts):
  for i in range(len(texts)):
    texts[i] = process_email(email.message_from_string(texts[i]))
  return texts

In [ ]:
sa_texts = get_processed_arr(sa_texts)

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.rebackee.com/cursos2/contraloria.htm
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.post-gazette.com/columnists/20020905brian5.asp



" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [ ]:
trec_texts = get_processed_arr(trec_texts)
trec1_texts, trec2_texts, trec1_classes, trec2_classes = train_test_split(trec_texts, trec_classes, test_size=0.5, random_state=42)
trec1_texts, trec3_texts, trec1_classes, trec3_classes = train_test_split(trec1_texts, trec1_classes, test_size=0.5, random_state=42)
trec4_texts, trec2_texts, trec4_classes, trec2_classes = train_test_split(trec2_texts, trec2_classes, test_size=0.5, random_state=42)
trec1_texts, trec5_texts, trec1_classes, trec5_classes = train_test_split(trec1_texts, trec1_classes, test_size=0.5, random_state=42)
trec6_texts, trec2_texts, trec6_classes, trec2_classes = train_test_split(trec2_texts, trec2_classes, test_size=0.5, random_state=42)
trec3_texts, trec7_texts, trec3_classes, trec7_classes = train_test_split(trec3_texts, trec3_classes, test_size=0.5, random_state=42)
trec4_texts, trec8_texts, trec4_classes, trec8_classes = train_test_split(trec4_texts, trec4_classes, test_size=0.5, random_state=42)

In [ ]:
ceas_texts = get_processed_arr(ceas_texts)
ceas1_texts, ceas2_texts, ceas1_classes, ceas2_classes = train_test_split(ceas_texts, ceas_classes, test_size=0.5, random_state=42)
ceas1_texts, ceas3_texts, ceas1_classes, ceas3_classes = train_test_split(ceas1_texts, ceas1_classes, test_size=0.5, random_state=42)
ceas2_texts, ceas4_texts, ceas2_classes, ceas4_classes = train_test_split(ceas2_texts, ceas2_classes, test_size=0.5, random_state=42)
ceas1_texts, ceas5_texts, ceas1_classes, ceas5_classes = train_test_split(ceas1_texts, ceas1_classes, test_size=0.5, random_state=42)
ceas2_texts, ceas6_texts, ceas2_classes, ceas6_classes = train_test_split(ceas2_texts, ceas2_classes, test_size=0.5, random_state=42)
ceas3_texts, ceas7_texts, ceas3_classes, ceas7_classes = train_test_split(ceas3_texts, ceas3_classes, test_size=0.5, random_state=42)
ceas4_texts, ceas8_texts, ceas4_classes, ceas8_classes = train_test_split(ceas4_texts, ceas4_classes, test_size=0.5, random_state=42)

In [ ]:
"""
Tokenize and lemmatize document
"""
def tokenize_and_lemmatize(raw_text):
    regex = re.compile('[^a-zA-Z ]')
    raw_text = regex.sub(' ', raw_text)
    tokens = nltk.word_tokenize(raw_text)
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]     
    return tokens

def get_tokens_arr(email_texts):
  tokens_arr = []
  for raw_text in email_texts:
    tokens_arr.append(tokenize_and_lemmatize(raw_text.lower()))
  return tokens_arr

In [ ]:
INF = 1000000000
N_FEATURES = 10000
UNIQUE_FEATURE_THR = 50


"""
Afgorithm TFDCR of feature selection with BOW
"""
def tfdcr(tokens, classes):
  features = np.unique(np.concatenate(tokens, axis=None))
  features_weight = np.zeros(len(features))
  leg_docs = np.array([tokens[i] for i in range(len(tokens)) if not classes[i]])
  spam_docs = np.array([tokens[i] for i in range(len(tokens)) if classes[i]])
  counter_leg = [Counter(email) for email in leg_docs]
  counter_spam = [Counter(email) for email in spam_docs]
  counter_all = [Counter(email) for email in tokens]
  for i, term in enumerate(features):
    term_freq_spam = 0
    term_freq_leg = 0
    doc_freq_spam = 0
    doc_freq_leg = 0
    for j in range(len(leg_docs)):
        if counter_leg[j][term] != 0:
            doc_freq_leg += 1
            term_freq_leg += counter_leg[j][term]
    for j in range(len(spam_docs)):
        if counter_spam[j][term] != 0:
            doc_freq_spam += 1
            term_freq_spam += counter_spam[j][term]
    if doc_freq_leg == 0 or doc_freq_spam == 0:
      if term_freq_leg + term_freq_spam >= UNIQUE_FEATURE_THR:
        features_weight[i] = INF
      else:
        features_weight[i] = 0
    else:
      product = doc_freq_spam / len(spam_docs) * len(leg_docs) / doc_freq_leg if doc_freq_spam / len(spam_docs) > doc_freq_leg / len(leg_docs) else doc_freq_leg / len(leg_docs) * len(spam_docs) / doc_freq_spam
      features_weight[i] = np.abs(term_freq_leg - term_freq_spam) * product
  ids = features_weight.argsort()
  best_features = features[ids[::-1]][:min(N_FEATURES, len(features)):]
  vectors = np.zeros((len(tokens), len(best_features)))
  for i in range(len(tokens)):
    for j in range(len(best_features)):
      vectors[i][j] = counter_all[i][best_features[j]]
  return vectors

In [ ]:
INF = 1000000000
N_FEATURES = 10000
UNIQUE_FEATURE_THR = 50


"""
Afgorithm TFDCR of feature selection with TFIDF
"""
def tfdcr(tokens, classes):
  features = np.unique(np.concatenate(tokens, axis=None))
  features_weight = np.zeros(len(features))
  leg_docs = np.array([tokens[i] for i in range(len(tokens)) if not classes[i]])
  spam_docs = np.array([tokens[i] for i in range(len(tokens)) if classes[i]])
  counter_leg = [Counter(email) for email in leg_docs]
  counter_spam = [Counter(email) for email in spam_docs]
  counter_all = [Counter(email) for email in tokens]
  for i, term in enumerate(features):
    term_freq_spam = 0
    term_freq_leg = 0
    doc_freq_spam = 0
    doc_freq_leg = 0
    for j in range(len(leg_docs)):
        if counter_leg[j][term] != 0:
            doc_freq_leg += 1
            term_freq_leg += counter_leg[j][term]
    for j in range(len(spam_docs)):
        if counter_spam[j][term] != 0:
            doc_freq_spam += 1
            term_freq_spam += counter_spam[j][term]
    if doc_freq_leg == 0 or doc_freq_spam == 0:
      if term_freq_leg + term_freq_spam >= UNIQUE_FEATURE_THR:
        features_weight[i] = INF
      else:
        features_weight[i] = 0
    else:
      product = doc_freq_spam / len(spam_docs) * len(leg_docs) / doc_freq_leg if doc_freq_spam / len(spam_docs) > doc_freq_leg / len(leg_docs) else doc_freq_leg / len(leg_docs) * len(spam_docs) / doc_freq_spam
      features_weight[i] = np.abs(term_freq_leg - term_freq_spam) * product
  ids = features_weight.argsort()
  best_features = features[ids[::-1]][:min(N_FEATURES, len(features)):]
  vectors = np.zeros((len(tokens), len(best_features)))
  for i in range(len(tokens)):
    for j in range(len(best_features)):
      vectors[i][j] = counter_all[i][best_features[j]]
  tf  = TfidfTransformer()
  return tf.fit_transform(vectors, classes)

In [ ]:
ALPHA = 0.5
BETA = 0.5

def find_best_solution(features, classes, whales, validation_X, validation_y):
  clf = KNeighborsClassifier()
  ans = None
  best_score = 0
  for whale in whales:
    print(1)
    cur_features = features[:, whale.astype(np.bool)]
    clf.fit(cur_features, classes)
    score = clf.score(validation_X[:, whale.astype(np.bool)], validation_y)
    score = ALPHA * (1 - score) + BETA * len(cur_features[0]) / len(features[0])
    if score < best_score:
      best_score = score
      ans = whale
  return ans


def woa(features, classes, n_whales, validation_X, validation_y, max_iters, B):
  whales = np.random.randint(0, 2, (n_whales, len(features[0])))
  prey = find_best_solution(features, classes, whales, validation_X, validation_y)
  for t in range(max_iters):
    print(t)
    a = 2 - 2 * t / max_iters
    for i in len(whales):
      r = np.random.random(len(whales[0]))
      print(r)
      A = 2 * a * r - a
      C = 2 * r
      if (np.random.random() >= 0.5):
        D = np.abs(whales[i] - prey)
        l = np.random.random() * 2.0 - 1
        whales[i] = D * np.exp(B * l) * np.cos(2 * np.pi * l) + prey
      elif np.linalg.norm(A) < 1:
        D = np.abs(C * prey - whales[i])
        whales[i] = prey - A * D
      else:
        rand_whale = whales[np.random.randint(0, n_whales)]
        D = np.abs(C * rand_whale - whales[i])
        whales[i] = rand_whale - A * D
      whales[i] = (whales[i] >= 0.5).astype('int')
    prey = find_best_solution(features, classes, whales, validation_X, validation_y)
  return features[:, prey.astype(np.bool)]

In [ ]:
def do_tests_with_woa(clf, texts, classes, out_prefix):
  with tf.device('/device:GPU:0'):
    cv = CountVectorizer(tokenizer=tmp, preprocessor=tmp)
    vecs = cv.fit_transform(get_tokens_arr(texts)).toarray()
    X, X_val, y, y_val = train_test_split(vecs, classes, test_size=0.33)
    print(woa(X, y, 10, X_val, y_val, 12, 1))

do_tests_with_woa(1, sa_texts, sa_classes, "")

KeyboardInterrupt: ignored

In [ ]:
def tmp(email):
  return email

def do_tests_with_clf(clf, texts, classes, out_prefix, n_features=N_FEATURES):
  with tf.device('/device:GPU:0'):
    cv = TfidfVectorizer(tokenizer=tmp, preprocessor=tmp)
    chi = SelectKBest(chi2, k=n_features)
    train_X, test_X, train_y, test_y = train_test_split(tfdcr(get_tokens_arr(texts), classes), classes * 2 - 1, test_size=0.33)
    clf.fit(train_X, train_y)
    print(("{} TFDCR: accuracy={} roc_auc={}").format(out_prefix, clf.score(test_X, test_y), roc_auc_score(test_y, clf.predict(test_X))))
    gc.collect()
    train_X, test_X, train_y, test_y = train_test_split(chi.fit_transform(cv.fit_transform(get_tokens_arr(texts)).toarray(), classes), classes * 2 - 1, test_size=0.33)
    gc.collect()
    clf.fit(train_X, train_y)
    print(("{} CHI: accuracy={} roc_auc={}").format(out_prefix, clf.score(test_X, test_y), roc_auc_score(test_y, clf.predict(test_X))))

In [ ]:
def tmp(email):
  return email

def do_tests_with_clf(clf, texts, classes, out_prefix):
  with tf.device('/device:GPU:0'):
    cv = TfidfVectorizer(tokenizer=tmp, preprocessor=tmp)
    gc.collect()
    train_X, test_X, train_y, test_y = train_test_split(cv.fit_transform(get_tokens_arr(texts)).toarray(), classes * 2 - 1, test_size=0.33)
    gc.collect()
    clf.fit(train_X, train_y)
    print(("{} NO SELEC: accuracy={} roc_auc={}").format(out_prefix, clf.score(test_X, test_y), roc_auc_score(test_y, clf.predict(test_X))))

In [ ]:
gc.collect()
do_tests_with_clf(cert, ceas1_texts, ceas1_classes, "CERT CEAS1", 4000)
gc.collect()
do_tests_with_clf(cert, ceas2_texts, ceas2_classes, "CERT CEAS2", 4000)
gc.collect()
do_tests_with_clf(cert, ceas3_texts, ceas3_classes, "CERT CEAS3", 4000)
gc.collect()
do_tests_with_clf(cert, ceas4_texts, ceas4_classes, "CERT CEAS4", 4000)
gc.collect()
do_tests_with_clf(cert, ceas5_texts, ceas5_classes, "CERT CEAS5", 4000)
gc.collect()
do_tests_with_clf(cert, ceas6_texts, ceas6_classes, "CERT CEAS6", 4000)
gc.collect()
do_tests_with_clf(cert, ceas7_texts, ceas7_classes, "CERT CEAS7", 4000)
gc.collect()
do_tests_with_clf(cert, ceas8_texts, ceas8_classes, "CERT CEAS8", 4000)
gc.collect()
do_tests_with_clf(svm, ceas1_texts, ceas1_classes, "SVM CEAS1", 4000)
gc.collect()
do_tests_with_clf(svm, ceas2_texts, ceas2_classes, "SVM CEAS2", 4000)

CERT CEAS1 TFDCR: accuracy=0.9838126399173411 roc_auc=0.9707950468013495
CERT CEAS1 CHI: accuracy=0.9860513173755812 roc_auc=0.9745333855405931
CERT CEAS2 TFDCR: accuracy=0.9820905803340796 roc_auc=0.9668466969198208
CERT CEAS2 CHI: accuracy=0.9795074909591872 roc_auc=0.9610256948356289
CERT CEAS3 TFDCR: accuracy=0.9788186671258825 roc_auc=0.9628254194417752
CERT CEAS3 CHI: accuracy=0.983640433959015 roc_auc=0.9677828835461677
CERT CEAS4 TFDCR: accuracy=0.9822627862924057 roc_auc=0.965562955274943
CERT CEAS4 CHI: accuracy=0.9834682280006888 roc_auc=0.9646555601757067
CERT CEAS5 TFDCR: accuracy=0.978133608815427 roc_auc=0.9621677927971
CERT CEAS5 CHI: accuracy=0.9801997245179064 roc_auc=0.9619135630554219
CERT CEAS6 TFDCR: accuracy=0.9807162534435262 roc_auc=0.9643228503711164
CERT CEAS6 CHI: accuracy=0.9762396694214877 roc_auc=0.9542896053492939
CERT CEAS7 TFDCR: accuracy=0.9819214876033058 roc_auc=0.967587288597927
CERT CEAS7 CHI: accuracy=0.9807162534435262 roc_auc=0.9618920648464163

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM CEAS1 TFDCR: accuracy=0.9213018770449457 roc_auc=0.7990570847529853


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM CEAS1 CHI: accuracy=0.9355949715860169 roc_auc=0.8369110739246884


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM CEAS2 TFDCR: accuracy=0.935767177544343 roc_auc=0.8420825125337268


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM CEAS2 CHI: accuracy=0.9483382125021526 roc_auc=0.8736012702898663


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [ ]:
cert = DecisionTreeClassifier()
svm = SVC(max_iter=1000)
gc.collect()
do_tests_with_clf(svm, ceas3_texts, ceas3_classes, "SVM CEAS3", 4000)
gc.collect()
do_tests_with_clf(svm, ceas4_texts, ceas4_classes, "SVM CEAS4", 4000)
gc.collect()
do_tests_with_clf(svm, ceas5_texts, ceas5_classes, "SVM CEAS5", 4000)
gc.collect()
do_tests_with_clf(svm, ceas6_texts, ceas6_classes, "SVM CEAS6", 4000)
gc.collect()
do_tests_with_clf(svm, ceas7_texts, ceas7_classes, "SVM CEAS7", 4000)
gc.collect()
do_tests_with_clf(svm, ceas8_texts, ceas8_classes, "SVM CEAS8", 4000)
gc.collect()

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM CEAS3 TFDCR: accuracy=0.9187187876700534 roc_auc=0.7959964154504137


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM CEAS3 CHI: accuracy=0.8978818667125883 roc_auc=0.7467593553708474


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


...

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM CEAS8 TFDCR: accuracy=0.9247589531680441 roc_auc=0.8089909318927262


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM CEAS8 CHI: accuracy=0.949896694214876 roc_auc=0.8777832424215328


0

In [ ]:
import warnings
warnings.filterwarnings('ignore')
gc.collect()
do_tests_with_clf(cert, trec1_texts, trec1_classes, "CERT trec1", 4000)
gc.collect()
do_tests_with_clf(cert, trec2_texts, trec2_classes, "CERT trec2", 4000)
gc.collect()
do_tests_with_clf(cert, trec3_texts, trec3_classes, "CERT trec3", 4000)
gc.collect()
do_tests_with_clf(cert, trec4_texts, trec4_classes, "CERT trec4", 4000)
gc.collect()
do_tests_with_clf(cert, trec5_texts, trec5_classes, "CERT trec5", 4000)
gc.collect()
do_tests_with_clf(cert, trec6_texts, trec6_classes, "CERT trec6", 4000)
gc.collect()
do_tests_with_clf(cert, trec7_texts, trec7_classes, "CERT trec7", 4000)
gc.collect()
do_tests_with_clf(cert, trec8_texts, trec8_classes, "CERT trec8", 4000)
gc.collect()
do_tests_with_clf(svm, trec1_texts, trec1_classes, "SVM trec1", 4000)
gc.collect()
do_tests_with_clf(svm, trec2_texts, trec2_classes, "SVM trec2", 4000)
gc.collect()
do_tests_with_clf(svm, trec3_texts, trec3_classes, "SVM trec3", 4000)
gc.collect()
do_tests_with_clf(svm, trec4_texts, trec4_classes, "SVM trec4", 4000)
gc.collect()
do_tests_with_clf(svm, trec5_texts, trec5_classes, "SVM trec5", 4000)
gc.collect()
do_tests_with_clf(svm, trec6_texts, trec6_classes, "SVM trec6", 4000)
gc.collect()
do_tests_with_clf(svm, trec7_texts, trec7_classes, "SVM trec7", 4000)
gc.collect()
do_tests_with_clf(svm, trec8_texts, trec8_classes, "SVM trec8", 4000)
gc.collect()

CERT trec1 TFDCR: accuracy=0.9733204757312761 roc_auc=0.9687742678982514
CERT trec1 CHI: accuracy=0.9736419157827065 roc_auc=0.9690414873759247
CERT trec2 TFDCR: accuracy=0.9720437017994858 roc_auc=0.966269607650745
CERT trec2 CHI: accuracy=0.968508997429306 roc_auc=0.9620780450780915
CERT trec3 TFDCR: accuracy=0.9736419157827065 roc_auc=0.9703921866046451
CERT trec3 CHI: accuracy=0.9726775956284153 roc_auc=0.9688475112364177
CERT trec4 TFDCR: accuracy=0.9697846351655416 roc_auc=0.9663106890068568
CERT trec4 CHI: accuracy=0.9665702346512376 roc_auc=0.9631042363663899
CERT trec5 TFDCR: accuracy=0.9723561555769848 roc_auc=0.9700750969235962
CERT trec5 CHI: accuracy=0.9710703953712633 roc_auc=0.9667798768838888
CERT trec6 TFDCR: accuracy=0.9614271938283511 roc_auc=0.9574492512113162
CERT trec6 CHI: accuracy=0.9781420765027322 roc_auc=0.9727658032249531
CERT trec7 TFDCR: accuracy=0.971401028277635 roc_auc=0.9664956746886972
CERT trec7 CHI: accuracy=0.9723650385604113 roc_auc=0.968192557299

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM trec1 TFDCR: accuracy=0.9614271938283511 roc_auc=0.9482043078171392


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM trec1 CHI: accuracy=0.9549983927997429 roc_auc=0.9388704250779583


...

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM trec8 TFDCR: accuracy=0.9550128534704371 roc_auc=0.9357805911923486


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVM trec8 CHI: accuracy=0.958547557840617 roc_auc=0.9417885127282272


0

In [ ]:
from sklearn.tree import DecisionTreeClassifier
cert = DecisionTreeClassifier()
do_tests_with_clf(cert, sa_texts, sa_classes, "CERT SA")
do_tests_with_clf(cert, en1t, en1c, "CERT EN1")
do_tests_with_clf(cert, en2t, en2c, "CERT EN2")
do_tests_with_clf(cert, en3t, en3c, "CERT EN3")
do_tests_with_clf(cert, en4t, en4c, "CERT EN4")
do_tests_with_clf(cert, en5t, en5c, "CERT EN5")
do_tests_with_clf(cert, en6t, en6c, "CERT EN6")

CERT SA TFDCR: accuracy=0.9148629148629148 roc_auc=0.916562569398179
CERT SA CHI: accuracy=0.9098124098124099 roc_auc=0.9068238236910424
CERT EN1 TFDCR: accuracy=0.9461042765084944 roc_auc=0.9300251404660266
CERT EN1 CHI: accuracy=0.9443468072642062 roc_auc=0.9331159900579952
CERT EN2 TFDCR: accuracy=0.9482669425763063 roc_auc=0.9292986741102582
CERT EN2 CHI: accuracy=0.950336264873254 roc_auc=0.9267090994813767
CERT EN3 TFDCR: accuracy=0.9593183067619572 roc_auc=0.9523456597493108
CERT EN3 CHI: accuracy=0.9494227597581089 roc_auc=0.9410338902552282
CERT EN4 TFDCR: accuracy=0.9611111111111111 roc_auc=0.9501817736149171
CERT EN4 CHI: accuracy=0.9621212121212122 roc_auc=0.9489493181222135
CERT EN5 TFDCR: accuracy=0.9478922716627635 roc_auc=0.9337558836114677
CERT EN5 CHI: accuracy=0.9654566744730679 roc_auc=0.9558877814691767
CERT EN6 TFDCR: accuracy=0.9454545454545454 roc_auc=0.9302696006135331
CERT EN6 CHI: accuracy=0.956060606060606 roc_auc=0.9386806138378166


In [ ]:
print("TFIDF!!!")
import warnings
warnings.filterwarnings('ignore')
gc.collect()
do_tests_with_clf(cert, ceas1_texts, ceas1_classes, "CERT CEAS1", 4000)
gc.collect()
do_tests_with_clf(cert, ceas2_texts, ceas2_classes, "CERT CEAS2", 4000)
gc.collect()
do_tests_with_clf(cert, ceas3_texts, ceas3_classes, "CERT CEAS3", 4000)
gc.collect()
do_tests_with_clf(cert, ceas4_texts, ceas4_classes, "CERT CEAS4", 4000)
gc.collect()
do_tests_with_clf(cert, ceas5_texts, ceas5_classes, "CERT CEAS5", 4000)
gc.collect()
do_tests_with_clf(cert, ceas6_texts, ceas6_classes, "CERT CEAS6", 4000)
gc.collect()
do_tests_with_clf(cert, ceas7_texts, ceas7_classes, "CERT CEAS7", 4000)
gc.collect()
do_tests_with_clf(cert, ceas8_texts, ceas8_classes, "CERT CEAS8", 4000)
gc.collect()
do_tests_with_clf(svm, ceas1_texts, ceas1_classes, "SVM CEAS1", 4000)
gc.collect()
do_tests_with_clf(svm, ceas2_texts, ceas2_classes, "SVM CEAS2", 4000)
gc.collect()
do_tests_with_clf(svm, ceas3_texts, ceas3_classes, "SVM CEAS3", 4000)
gc.collect()
do_tests_with_clf(svm, ceas4_texts, ceas4_classes, "SVM CEAS4", 4000)
gc.collect()
do_tests_with_clf(svm, ceas5_texts, ceas5_classes, "SVM CEAS5", 4000)
gc.collect()
do_tests_with_clf(svm, ceas6_texts, ceas6_classes, "SVM CEAS6", 4000)
gc.collect()
do_tests_with_clf(svm, ceas7_texts, ceas7_classes, "SVM CEAS7", 4000)
gc.collect()
do_tests_with_clf(svm, ceas8_texts, ceas8_classes, "SVM CEAS8", 4000)
gc.collect()

TFIDF!!!
CERT CEAS1 TFDCR: accuracy=0.9843292577923196 roc_auc=0.9709318862974178
CERT CEAS1 CHI: accuracy=0.9857069054589289 roc_auc=0.9722357983029754
CERT CEAS2 TFDCR: accuracy=0.9815739624591011 roc_auc=0.9630457192279269
CERT CEAS2 CHI: accuracy=0.9796796969175133 roc_auc=0.9578471960163945
CERT CEAS3 TFDCR: accuracy=0.9860513173755812 roc_auc=0.9733015397470779
CERT CEAS3 CHI: accuracy=0.9850180816256242 roc_auc=0.9721171945470973
CERT CEAS4 TFDCR: accuracy=0.9845014637506457 roc_auc=0.9696672052403558
CERT CEAS4 CHI: accuracy=0.9838126399173411 roc_auc=0.9645059617165208
CERT CEAS5 TFDCR: accuracy=0.9836432506887053 roc_auc=0.9680843719162887
CERT CEAS5 CHI: accuracy=0.9803719008264463 roc_auc=0.962473902221844
CERT CEAS6 TFDCR: accuracy=0.9841597796143251 roc_auc=0.9718811980618112
CERT CEAS6 CHI: accuracy=0.9822658402203857 roc_auc=0.9669734222542961
CERT CEAS7 TFDCR: accuracy=0.9793388429752066 roc_auc=0.9606083918099138
CERT CEAS7 CHI: accuracy=0.984331955922865 roc_auc=0.97

0

In [ ]:
print("TFIDF!!!")
import warnings
warnings.filterwarnings('ignore')
gc.collect()
do_tests_with_clf(cert, trec1_texts, trec1_classes, "CERT trec1", 4000)
gc.collect()
do_tests_with_clf(cert, trec2_texts, trec2_classes, "CERT trec2", 4000)
gc.collect()
do_tests_with_clf(cert, trec3_texts, trec3_classes, "CERT trec3", 4000)
gc.collect()
do_tests_with_clf(cert, trec4_texts, trec4_classes, "CERT trec4", 4000)
gc.collect()
do_tests_with_clf(cert, trec5_texts, trec5_classes, "CERT trec5", 4000)
gc.collect()
do_tests_with_clf(cert, trec6_texts, trec6_classes, "CERT trec6", 4000)
gc.collect()
do_tests_with_clf(cert, trec7_texts, trec7_classes, "CERT trec7", 4000)
gc.collect()
do_tests_with_clf(cert, trec8_texts, trec8_classes, "CERT trec8", 4000)
gc.collect()
do_tests_with_clf(svm, trec1_texts, trec1_classes, "SVM trec1", 4000)
gc.collect()
do_tests_with_clf(svm, trec2_texts, trec2_classes, "SVM trec2", 4000)
gc.collect()
do_tests_with_clf(svm, trec3_texts, trec3_classes, "SVM trec3", 4000)
gc.collect()
do_tests_with_clf(svm, trec4_texts, trec4_classes, "SVM trec4", 4000)
gc.collect()
do_tests_with_clf(svm, trec5_texts, trec5_classes, "SVM trec5", 4000)
gc.collect()
do_tests_with_clf(svm, trec6_texts, trec6_classes, "SVM trec6", 4000)
gc.collect()
do_tests_with_clf(svm, trec7_texts, trec7_classes, "SVM trec7", 4000)
gc.collect()
do_tests_with_clf(svm, trec8_texts, trec8_classes, "SVM trec8", 4000)
gc.collect()

TFIDF!!!
CERT trec1 TFDCR: accuracy=0.9672131147540983 roc_auc=0.965168724959729
CERT trec1 CHI: accuracy=0.9771777563484411 roc_auc=0.9737879301179924
CERT trec2 TFDCR: accuracy=0.9717223650385605 roc_auc=0.965083790360999
CERT trec2 CHI: accuracy=0.9726863753213367 roc_auc=0.9689211134088811
CERT trec3 TFDCR: accuracy=0.9707489553198329 roc_auc=0.9667835871227592
CERT trec3 CHI: accuracy=0.9726775956284153 roc_auc=0.9653634624581539
CERT trec4 TFDCR: accuracy=0.9652844744455159 roc_auc=0.9602601169573156
CERT trec4 CHI: accuracy=0.9723561555769848 roc_auc=0.9687055548178235
CERT trec5 TFDCR: accuracy=0.9723561555769848 roc_auc=0.9680402345009207
CERT trec5 CHI: accuracy=0.9710703953712633 roc_auc=0.9659855760432755
CERT trec6 TFDCR: accuracy=0.970106075216972 roc_auc=0.9662496898557281
CERT trec6 CHI: accuracy=0.9784635165541626 roc_auc=0.9750911067765959
CERT trec7 TFDCR: accuracy=0.9762210796915167 roc_auc=0.972372326092388
CERT trec7 CHI: accuracy=0.9749357326478149 roc_auc=0.9675

0

In [ ]:
print("TFIDF!!!!")
do_tests_with_clf(cert, sa_texts, sa_classes, "CERT SA")
do_tests_with_clf(cert, en1t, en1c, "CERT EN1")
do_tests_with_clf(cert, en2t, en2c, "CERT EN2")
do_tests_with_clf(cert, en3t, en3c, "CERT EN3")
do_tests_with_clf(cert, en4t, en4c, "CERT EN4")
do_tests_with_clf(cert, en5t, en5c, "CERT EN5")
do_tests_with_clf(cert, en6t, en6c, "CERT EN6")

TFIDF!!!!
CERT SA TFDCR: accuracy=0.9227994227994228 roc_auc=0.9186622823836182
CERT SA CHI: accuracy=0.911976911976912 roc_auc=0.9114573614480314
CERT EN1 TFDCR: accuracy=0.9466900995899239 roc_auc=0.9382305099525431
CERT EN1 CHI: accuracy=0.9548916227299356 roc_auc=0.9475346961643143
CERT EN2 TFDCR: accuracy=0.9534402483186757 roc_auc=0.9393234147469858
CERT EN2 CHI: accuracy=0.958613554061045 roc_auc=0.9438973199870844
CERT EN3 TFDCR: accuracy=0.9494227597581089 roc_auc=0.9314440115621465
CERT EN3 CHI: accuracy=0.9505222649807586 roc_auc=0.9337441957852471
CERT EN4 TFDCR: accuracy=0.9691919191919192 roc_auc=0.9600360083630577
CERT EN4 CHI: accuracy=0.9727272727272728 roc_auc=0.9577072270553233
CERT EN5 TFDCR: accuracy=0.9601873536299765 roc_auc=0.9525132450331125
CERT EN5 CHI: accuracy=0.9648711943793911 roc_auc=0.9570724841660803
CERT EN6 TFDCR: accuracy=0.9484848484848485 roc_auc=0.931372549019608
CERT EN6 CHI: accuracy=0.9580808080808081 roc_auc=0.9385646535036779


In [ ]:
import warnings
warnings.filterwarnings('ignore')
print("TFIDF!!!!")
do_tests_with_clf(svm, sa_texts, sa_classes, "SVC SA")
do_tests_with_clf(svm, en1t, en1c, "SVC EN1")
do_tests_with_clf(svm, en2t, en2c, "SVC EN2")
do_tests_with_clf(svm, en3t, en3c, "SVC EN3")
do_tests_with_clf(svm, en4t, en4c, "SVC EN4")
do_tests_with_clf(svm, en5t, en5c, "SVC EN5")
do_tests_with_clf(svm, en6t, en6c, "SVC EN6")

TFIDF!!!!
SVC SA TFDCR: accuracy=0.9718614718614719 roc_auc=0.9627271855087995
SVC SA CHI: accuracy=0.9754689754689755 roc_auc=0.9664473357324872
SVC EN1 TFDCR: accuracy=0.9818394844756884 roc_auc=0.9818654269684197
SVC EN1 CHI: accuracy=0.9876977152899824 roc_auc=0.9882661353389595
SVC EN2 TFDCR: accuracy=0.9870667356440765 roc_auc=0.9775829487723718
SVC EN2 CHI: accuracy=0.9948266942576306 roc_auc=0.9911112371660332
SVC EN3 TFDCR: accuracy=0.9950522264980759 roc_auc=0.9925929570996073
SVC EN3 CHI: accuracy=0.9934029686641012 roc_auc=0.9930992860822846
SVC EN4 TFDCR: accuracy=0.9792929292929293 roc_auc=0.9591633466135457
SVC EN4 CHI: accuracy=0.9828282828282828 roc_auc=0.965931863727455
SVC EN5 TFDCR: accuracy=0.9894613583138173 roc_auc=0.9829222011385199
SVC EN5 CHI: accuracy=0.9929742388758782 roc_auc=0.9871794871794871
SVC EN6 TFDCR: accuracy=0.9803030303030303 roc_auc=0.9610460923157618
SVC EN6 CHI: accuracy=0.9863636363636363 roc_auc=0.9764728826967872


In [ ]:
import warnings
warnings.filterwarnings('ignore')
print("TFIDF!!!!")
gc.collect()
do_tests_with_clf(rf, en1t, en1c, "RotF EN1")
gc.collect()
do_tests_with_clf(rf, en2t, en2c, "RotF EN2")
gc.collect()
do_tests_with_clf(rf, en3t, en3c, "RotF EN3")
gc.collect()
do_tests_with_clf(rf, en4t, en4c, "RotF EN4")
gc.collect()
do_tests_with_clf(rf, en5t, en5c, "RotF EN5")
gc.collect()
do_tests_with_clf(rf, en6t, en6c, "RotF EN6")
gc.collect()
do_tests_with_clf(rf, sa_texts, sa_classes, "RotF SA")

TFIDF!!!!


TypeError: ignored

In [ ]:
svm = SVC(max_iter=1000)
do_tests_with_clf(svm, sa_texts, sa_classes, "SVC SA")
do_tests_with_clf(svm, en1t, en1c, "SVC EN1")
do_tests_with_clf(svm, en2t, en2c, "SVC EN2")
do_tests_with_clf(svm, en3t, en3c, "SVC EN3")
do_tests_with_clf(svm, en4t, en4c, "SVC EN4")
do_tests_with_clf(svm, en5t, en5c, "SVC EN5")
do_tests_with_clf(svm, en6t, en6c, "SVC EN6")

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC SA TFDCR: accuracy=0.8614718614718615 roc_auc=0.7796778683634918


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC SA CHI: accuracy=0.8210678210678211 roc_auc=0.7353896103896104


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


...

SVC EN5 CHI: accuracy=0.9613583138173302 roc_auc=0.9345860927152317


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC EN6 TFDCR: accuracy=0.9474747474747475 roc_auc=0.896414342629482


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVC EN6 CHI: accuracy=0.944949494949495 roc_auc=0.8918955172918395


In [ ]:
import warnings
warnings.filterwarnings('ignore')
rf = RotationForestClassifier(max_depth=50, n_estimators=5)
gc.collect()
do_tests_with_clf(rf, sa_texts, sa_classes, "RotF SA")
gc.collect()
do_tests_with_clf(rf, en1t, en1c, "RotF EN1")
gc.collect()
do_tests_with_clf(rf, en2t, en2c, "RotF EN2")
gc.collect()
do_tests_with_clf(rf, en3t, en3c, "RotF EN3")
gc.collect()
do_tests_with_clf(rf, en4t, en4c, "RotF EN4")
gc.collect()
do_tests_with_clf(rf, en5t, en5c, "RotF EN5")
gc.collect()
do_tests_with_clf(rf, en6t, en6c, "RotF EN6")
gc.collect()

RotF SA TFDCR: accuracy=0.9242424242424242 roc_auc=0.913201530612245
RotF SA CHI: accuracy=0.9307359307359307 roc_auc=0.9216903612886397
RotF EN1 TFDCR: accuracy=0.9402460456942003 roc_auc=0.9290880476100382
RotF EN1 CHI: accuracy=0.9414176918570591 roc_auc=0.9366005701677309
RotF EN2 TFDCR: accuracy=0.9622348680807036 roc_auc=0.9489115893095053
RotF EN2 CHI: accuracy=0.9580962234868081 roc_auc=0.9461866810369278
RotF EN3 TFDCR: accuracy=0.9532710280373832 roc_auc=0.9426020018920321
RotF EN3 CHI: accuracy=0.954370533260033 roc_auc=0.9399778128536542
RotF EN4 TFDCR: accuracy=0.9666666666666667 roc_auc=0.9552712944327998
RotF EN4 CHI: accuracy=0.9626262626262626 roc_auc=0.9457018356828123
RotF EN5 TFDCR: accuracy=0.9490632318501171 roc_auc=0.9362657712155256
RotF EN5 CHI: accuracy=0.9648711943793911 roc_auc=0.95496136936014
RotF EN6 TFDCR: accuracy=0.95 roc_auc=0.9333677685950413
RotF EN6 CHI: accuracy=0.9570707070707071 roc_auc=0.9296609139523578


72

In [ ]:
import warnings
warnings.filterwarnings('ignore')
rf = RotationForestClassifier(max_depth=10, n_estimators=2)
gc.collect()
do_tests_with_clf(rf, trec1_texts, trec1_classes, "RotF trec1", 500)
gc.collect()
do_tests_with_clf(rf, trec2_texts, trec2_classes, "RotF trec2", 500)
gc.collect()
do_tests_with_clf(rf, trec3_texts, trec3_classes, "RotF trec3", 500)
gc.collect()
do_tests_with_clf(rf, trec4_texts, trec4_classes, "RotF trec4", 500)
gc.collect()
do_tests_with_clf(rf, trec5_texts, trec5_classes, "RotF trec5", 500)
gc.collect()
do_tests_with_clf(rf, trec6_texts, trec6_classes, "RotF trec6", 500)
gc.collect()
do_tests_with_clf(rf, trec7_texts, trec7_classes, "RotF trec7", 500)

RotF trec1 TFDCR: accuracy=0.96849887495982 roc_auc=0.960286287293758
RotF trec1 CHI: accuracy=0.9704275152684024 roc_auc=0.9634288077143954
RotF trec2 TFDCR: accuracy=0.9691516709511568 roc_auc=0.9599837343588432
RotF trec2 CHI: accuracy=0.9611182519280206 roc_auc=0.9489333021183456
RotF trec3 TFDCR: accuracy=0.9694631951141112 roc_auc=0.9601905719144925
RotF trec3 CHI: accuracy=0.9691417550626809 roc_auc=0.9591595247055299
RotF trec4 TFDCR: accuracy=0.9681774349083896 roc_auc=0.958836641050511
RotF trec4 CHI: accuracy=0.9646415943426551 roc_auc=0.9560103534206215
RotF trec5 TFDCR: accuracy=0.9717132754741241 roc_auc=0.9665814606301064
RotF trec5 CHI: accuracy=0.9736419157827065 roc_auc=0.9681489262371615
RotF trec6 TFDCR: accuracy=0.9742847958855674 roc_auc=0.9690526037879174
RotF trec6 CHI: accuracy=0.9672131147540983 roc_auc=0.959038737446198
RotF trec7 TFDCR: accuracy=0.9707583547557841 roc_auc=0.964387771431184
RotF trec7 CHI: accuracy=0.971401028277635 roc_auc=0.9648186194346742

In [ ]:
gc.collect()
do_tests_with_clf(rf, trec8_texts, trec8_classes, "RotF trec8", 500)

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.6/dist-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_pca.py:456: RuntimeWarning: invalid value encountered in true_divide
  explained_variance

RotF trec8 TFDCR: accuracy=0.9749357326478149 roc_auc=0.9694752326553007


/usr/local/lib/python3.6/dist-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  FutureWarning)


RotF trec8 CHI: accuracy=0.9691516709511568 roc_auc=0.9619592763766507


In [ ]:
do_tests_with_clf(cert, sa_texts, sa_classes, "CERT SA")
do_tests_with_clf(cert, en1t, en1c, "CERT EN1")
do_tests_with_clf(cert, en2t, en2c, "CERT EN2")
do_tests_with_clf(cert, en3t, en3c, "CERT EN3")
do_tests_with_clf(cert, en4t, en4c, "CERT EN4")
do_tests_with_clf(cert, en5t, en5c, "CERT EN5")
do_tests_with_clf(cert, en6t, en6c, "CERT EN6")

CERT SA NO SELEC: accuracy=0.9083694083694084 roc_auc=0.904551896270914
CERT EN1 NO SELEC: accuracy=0.9338019917984769 roc_auc=0.9321553093434138
CERT EN2 NO SELEC: accuracy=0.9539575788929125 roc_auc=0.9379009936073378
CERT EN3 NO SELEC: accuracy=0.9571192963166575 roc_auc=0.942794613935042
CERT EN4 NO SELEC: accuracy=0.9702020202020202 roc_auc=0.9573456898059102
CERT EN5 NO SELEC: accuracy=0.9590163934426229 roc_auc=0.9485573299924638
CERT EN6 NO SELEC: accuracy=0.9489898989898989 roc_auc=0.9282407407407407


In [ ]:
import warnings
warnings.filterwarnings('ignore')
do_tests_with_clf(svm, sa_texts, sa_classes, "SVC SA")
do_tests_with_clf(svm, en1t, en1c, "SVC EN1")
do_tests_with_clf(svm, en2t, en2c, "SVC EN2")
do_tests_with_clf(svm, en3t, en3c, "SVC EN3")
do_tests_with_clf(svm, en4t, en4c, "SVC EN4")
do_tests_with_clf(svm, en5t, en5c, "SVC EN5")
do_tests_with_clf(svm, en6t, en6c, "SVC EN6")

SVC SA NO SELEC: accuracy=0.9632034632034632 roc_auc=0.9526146194529088
SVC EN1 NO SELEC: accuracy=0.9882835383714118 roc_auc=0.9882958633533485
SVC EN2 NO SELEC: accuracy=0.9901707190894982 roc_auc=0.981712001142286
SVC EN3 NO SELEC: accuracy=0.994502473886751 roc_auc=0.9904030311241394
SVC EN4 NO SELEC: accuracy=0.9787878787878788 roc_auc=0.9542483660130718
SVC EN5 NO SELEC: accuracy=0.984192037470726 roc_auc=0.9735294117647059
SVC EN6 NO SELEC: accuracy=0.9752525252525253 roc_auc=0.950476339735846


In [ ]:
import warnings
warnings.filterwarnings('ignore')
gc.collect()
do_tests_with_clf(cert, trec1_texts, trec1_classes, "CERT trec1")
gc.collect()
do_tests_with_clf(cert, trec2_texts, trec2_classes, "CERT trec2")
gc.collect()
do_tests_with_clf(cert, trec3_texts, trec3_classes, "CERT trec3")
gc.collect()
do_tests_with_clf(cert, trec4_texts, trec4_classes, "CERT trec4")
gc.collect()
do_tests_with_clf(cert, trec5_texts, trec5_classes, "CERT trec5")
gc.collect()
do_tests_with_clf(cert, trec6_texts, trec6_classes, "CERT trec6")
gc.collect()
do_tests_with_clf(cert, trec7_texts, trec7_classes, "CERT trec7")
gc.collect()
do_tests_with_clf(cert, trec8_texts, trec8_classes, "CERT trec8")
gc.collect()
do_tests_with_clf(svm, trec1_texts, trec1_classes, "SVM trec1")
gc.collect()
do_tests_with_clf(svm, trec2_texts, trec2_classes, "SVM trec2")
gc.collect()
do_tests_with_clf(svm, trec3_texts, trec3_classes, "SVM trec3")
gc.collect()
do_tests_with_clf(svm, trec4_texts, trec4_classes, "SVM trec4")
gc.collect()
do_tests_with_clf(svm, trec5_texts, trec5_classes, "SVM trec5")
gc.collect()
do_tests_with_clf(svm, trec6_texts, trec6_classes, "SVM trec6")
gc.collect()
do_tests_with_clf(svm, trec7_texts, trec7_classes, "SVM trec7")
gc.collect()
do_tests_with_clf(svm, trec8_texts, trec8_classes, "SVM trec8")